### Import Tensorflow and Numpy packages

In [12]:
import tensorflow as tf
import numpy as np
print("Tensorflow version: ", tf.__version__)
print("Numpy version     : ", np.__version__)

Tensorflow version:  1.10.0
Numpy version     :  1.15.2


### Download MNIST digit dataset
The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

##### Process Data - Reshape mnist dataset for input node.

In [3]:
x_train = x_train.reshape([-1, 784])
x_train = x_train.astype(float)
y_train = y_train.astype(np.int32)

x_test = x_test.reshape([-1, 784])
x_test = x_test.astype(float)
y_test = y_test.astype(np.int32)

##### Check the data

In [4]:
# 60000 data samples
print("Train Samples: {}".format(x_train.shape))
print("Train Outputs: {}".format(y_train.shape))

# Sample size 28x28
# 10000 test samples
print("Eval Samples: {}".format(x_test.shape))
print("Eval Outputs: {}".format(y_test.shape))

Train Samples: (60000, 784)
Train Outputs: (60000,)
Eval Samples: (10000, 784)
Eval Outputs: (10000,)


### Write a model function
###### Model function runs in 3 different modes: Train, Evaluate and Test.
To implement a typical model function, you must do the following:
  - Define the model.
  - Specify additional calculations for each of the three different modes:
      - Predict
      - Evaluate
      - Train

In [5]:
def cnn_model_fn(features, labels, mode):
    
    input_layer = features["x"]
    fc1 = tf.layers.dense(input_layer, 256, activation=tf.nn.relu, name="Hidden-1")
    fc2 = tf.layers.dense(fc1, 128, activation=tf.nn.relu, name="Hidden-2")
    dropout = tf.layers.dropout(inputs=fc2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1, name="classes_op"),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels, predictions= predictions["classes"], name='acc_op')
    metrics = {'accuracy': accuracy}

    # merging and saving them every 100 steps by default for Tensorboard
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    tf.summary.scalar('loss', loss)

    internal_logging_hook = tf.train.LoggingTensorHook({"1_loss": loss, "2_accuracy": accuracy[1]}, every_n_iter=10 )

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op, training_hooks = [internal_logging_hook])

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


##### Create the Estimator

In [6]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                          model_dir="./checkpoints/log2jupyter",
                                          config=tf.estimator.RunConfig(log_step_count_steps=50,
                                                                        save_summary_steps=50),)


INFO:tensorflow:Using config: {'_model_dir': './checkpoints/log2jupyter', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000232B140B908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### Prepare input function for Training

In [7]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_train},
                                                    y=y_train,
                                                    batch_size=100,
                                                    num_epochs=None,
                                                    shuffle=True)


##### Train .. Train .. Train ..

In [8]:
# Train the model with external logging hook
mnist_classifier.train(input_fn=train_input_fn, steps=2000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/log2jupyter\model.ckpt-65000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 65000 into ./checkpoints/log2jupyter\model.ckpt.
INFO:tensorflow:loss = 0.08926447480916977, step = 65001
INFO:tensorflow:1_loss = 0.08926447480916977, 2_accuracy = 0.97
INFO:tensorflow:1_loss = 0.0753067210316658, 2_accuracy = 0.97 (0.213 sec)
INFO:tensorflow:1_loss = 0.061375636607408524, 2_accuracy = 0.9766667 (0.060 sec)
INFO:tensorflow:1_loss = 0.1067533865571022, 2_accuracy = 0.9725 (0.054 sec)
INFO:tensorflow:1_loss = 0.09109072387218475, 2_accuracy = 0.974 (0.067 sec)
INFO:tensorflow:global_step/sec: 112.454
INFO:tensorflow:loss = 0.11332997679710388, step = 65051 (0.445 sec)
INFO:tensorflow:1_loss = 0.11332997679710388, 2_

INFO:tensorflow:1_loss = 0.14945606887340546, 2_accuracy = 0.9635065 (0.077 sec)
INFO:tensorflow:1_loss = 0.0657038539648056, 2_accuracy = 0.96371794 (0.070 sec)
INFO:tensorflow:1_loss = 0.20590510964393616, 2_accuracy = 0.96303797 (0.056 sec)
INFO:tensorflow:1_loss = 0.04821448400616646, 2_accuracy = 0.96325 (0.058 sec)
INFO:tensorflow:global_step/sec: 152.441
INFO:tensorflow:loss = 0.10304991155862808, step = 65801 (0.329 sec)
INFO:tensorflow:1_loss = 0.10304991155862808, 2_accuracy = 0.9633333 (0.066 sec)
INFO:tensorflow:1_loss = 0.12054808437824249, 2_accuracy = 0.9634146 (0.062 sec)
INFO:tensorflow:1_loss = 0.10004724562168121, 2_accuracy = 0.9633735 (0.064 sec)
INFO:tensorflow:1_loss = 0.11485181748867035, 2_accuracy = 0.9632143 (0.059 sec)
INFO:tensorflow:1_loss = 0.06084318459033966, 2_accuracy = 0.9632941 (0.059 sec)
INFO:tensorflow:global_step/sec: 163.404
INFO:tensorflow:loss = 0.1444626897573471, step = 65851 (0.305 sec)
INFO:tensorflow:1_loss = 0.1444626897573471, 2_accura

INFO:tensorflow:1_loss = 0.0928867906332016, 2_accuracy = 0.963121 (0.091 sec)
INFO:tensorflow:1_loss = 0.09521424770355225, 2_accuracy = 0.96303797 (0.045 sec)
INFO:tensorflow:1_loss = 0.04264884814620018, 2_accuracy = 0.96320754 (0.076 sec)
INFO:tensorflow:1_loss = 0.11551745235919952, 2_accuracy = 0.9631875 (0.059 sec)
INFO:tensorflow:global_step/sec: 149.411
INFO:tensorflow:loss = 0.10982440412044525, step = 66601 (0.334 sec)
INFO:tensorflow:1_loss = 0.10982440412044525, 2_accuracy = 0.9631056 (0.062 sec)
INFO:tensorflow:1_loss = 0.03110559470951557, 2_accuracy = 0.96320987 (0.062 sec)
INFO:tensorflow:1_loss = 0.1814202070236206, 2_accuracy = 0.96300614 (0.076 sec)
INFO:tensorflow:1_loss = 0.12711142003536224, 2_accuracy = 0.9629878 (0.063 sec)
INFO:tensorflow:1_loss = 0.06910046190023422, 2_accuracy = 0.9630303 (0.051 sec)
INFO:tensorflow:global_step/sec: 153.813
INFO:tensorflow:loss = 0.11660802364349365, step = 66651 (0.326 sec)
INFO:tensorflow:1_loss = 0.11660802364349365, 2_ac

##### Prepare input function for Evaluation

In [9]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x_test},
                                                   y=y_test,
                                                   num_epochs=1,
                                                   shuffle=False)

##### Evaluate

In [10]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-10-18-08:42:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoints/log2jupyter\model.ckpt-67000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-10-18-08:42:08
INFO:tensorflow:Saving dict for global step 67000: accuracy = 0.9662, global_step = 67000, loss = 0.18059248
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 67000: ./checkpoints/log2jupyter\model.ckpt-67000


### Final result

In [11]:
print(eval_results)

{'accuracy': 0.9662, 'loss': 0.18059248, 'global_step': 67000}
